# COVID-19 Documents Data Preprocessing

This Jupyter notebook demonstrates how to preprocess COVID-19 article data by using python code. Data preprocessing aims to make data to be useful for analysis, which contains removing duplications, non-English documents, cleaning text, and reformatting data table. 

In [2]:
import pandas as pd
import re
import json
import nltk
from langdetect import detect
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
INPUT_PATH = '../data/original data/covid-json/'
OUTPUT_PATH = '../data/outputs/articles/'

## Load Data

In [4]:
# Use pandas library to convert json to dataframe

df1 = pd.read_json(INPUT_PATH+"covid_19.json")
df2 = pd.read_json(INPUT_PATH+"covid19.json")
df3 = pd.read_json(INPUT_PATH+"sars_cov_2.json")
print(df1.shape, df2.shape, df3.shape)

(405248, 71) (107277, 71) (86873, 68)


In [5]:
# Combine the converted dataframes

frames = [df1, df2, df3]
covid_df = pd.concat(frames)

# Remove duplicated articles by '_id'
covid_df.drop_duplicates(subset=['_id'], keep='first', inplace=True)
covid_df.shape

(480297, 71)

In [6]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480297 entries, 0 to 86870
Data columns (total 71 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   _id                     480297 non-null  object 
 1   abstract                245584 non-null  object 
 2   URL                     480297 non-null  object 
 3   member                  480112 non-null  float64
 4   score                   480297 non-null  int64  
 5   created                 480297 non-null  object 
 6   update-policy           147550 non-null  object 
 7   license                 259697 non-null  object 
 8   ISSN                    389019 non-null  object 
 9   container-title         416929 non-null  object 
 10  issued                  480297 non-null  object 
 11  prefix                  480297 non-null  float64
 12  reference-count         480297 non-null  int64  
 13  indexed                 480297 non-null  object 
 14  author               

In [7]:
def date_formatting(date_str):
    date_str = str(date_str).replace("'", '"')
    date = json.loads(date_str)
    row_value = date['date-parts']
    year = row_value[0][0]
    month = row_value[0][1]
    day = row_value[0][2]
    return "{}/{}/{}".format(month, day, year)

In [8]:
# Timeframe of created date
sub_df = covid_df[['_id', 'created']]

sub_df['date'] = sub_df['created'].apply(lambda x: date_formatting(x))
sub_df['date'] = pd.to_datetime(sub_df['date']).dt.to_period('D')
sub_df.sort_values(by='date')

,_id,created,date
54965,10.1016/0168-1702(95)00046-s,"{'date-parts': [[2002, 7, 25]], 'date-time': '...",2002-07-25
49291,10.1016/s0168-1702(01)00432-4,"{'date-parts': [[2002, 7, 25]], 'date-time': '...",2002-07-25
40764,10.1016/s0929-693x(01)00696-0,"{'date-parts': [[2002, 7, 25]], 'date-time': '...",2002-07-25
56200,10.1016/s0165-2478(97)87550-5,"{'date-parts': [[2002, 7, 25]], 'date-time': '...",2002-07-25
31251,10.1016/s0966-842x(00)89018-6,"{'date-parts': [[2002, 7, 25]], 'date-time': '...",2002-07-25
...,...,...,...
344664,10.1016/j.ijid.2021.12.146,"{'date-parts': [[2022, 2, 28]], 'date-time': '...",2022-02-28
344665,10.36106/ijar/7000796,"{'date-parts': [[2022, 2, 28]], 'date-time': '...",2022-02-28
344666,10.1016/j.rceng.2021.11.005,"{'date-parts': [[2022, 2, 28]], 'date-time': '...",2022-02-28
344668,10.22541/au.164608823.39016270/v1,"{'date-parts': [[2022, 2, 28]], 'date-time': '...",2022-02-28


## Exclude non-English articles

In [9]:
def detect_language(text):
    try:
        language = detect(text)
    except:
        language = 'Error'
    return language

In [10]:
covid_df['text'] = covid_df['title'].astype(str) + ' ' + covid_df['abstract'].astype(str) + ' ' + covid_df['pubmed-abstract'].astype(str)
covid_df['language'] = covid_df['text'].apply(detect_language)
english_covid_df = covid_df.loc[covid_df['language'] == 'en']

In [11]:
# save English COVID articles

cols = list(english_covid_df.columns)
cols.remove('text')
cols.remove('language')
english_covid_df[cols].to_csv(OUTPUT_PATH + "covid_articles.tsv", sep='\t', encoding='utf-8', index=False)
english_covid_df.shape

(414062, 73)

## Cleaning text

In [12]:
def cleanhtml(raw_html):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    cleantext = re.sub('  ', ' ', cleantext)
    return cleantext


def cleaning_text(text):
    """
        Remove stop-words
        No digits
        No word length less than 3 
        Convert to lowercase
    """
    # remove html tags
    cleantext = cleanhtml(text)
    lemmatizer = WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))

    pos_family = {
        'noun': ['NN', 'NNS', 'NNP', 'NNPS'],
        'pron': ['PRP', 'PRP$', 'WP', 'WP$'],
        'verb': ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'],
        'adj': ['JJ', 'JJR', 'JJS'],
        'adv': ['RB', 'RBR', 'RBS', 'WRB']
    }
    
    regex = r"\b[^\d\W]+\b"
    tokens = []
    cleantext = cleantext.replace('-', '_')
    sentences = nltk.sent_tokenize(cleantext)
    
    for s in sentences:
        words = re.findall(regex, s)
        pairs = nltk.pos_tag(words)
        for pair in pairs:
            w = list(pair)[0] 
            tag = list(pair)[1]
            if w in stopwords: continue
            if w.isdigit(): continue
            if w.isupper() != True:w = w.lower() 
            if len(w) <= 3: continue
            if tag in pos_family['noun']:
                w = lemmatizer.lemmatize(w, 'n')
            elif tag in pos_family['pron']: # e.g their, self, what
                w = lemmatizer.lemmatize(w)
            elif tag in pos_family['verb']: # e.g experienced, based, evaluating, trying, healthcare
                w = lemmatizer.lemmatize(w, 'v')
            elif tag in pos_family['adj']: # e.g significant, pandemic, clinical, sensitive
                w = lemmatizer.lemmatize(w, 'a')
            elif tag in pos_family['adv']: #e.g. sore, seriously, alone, nationally
                w = lemmatizer.lemmatize(w, 'r')
            tokens.append(w)
    cleaned_text = ' '.join(tokens)
 
    return cleaned_text

In [13]:
def removing_stopwords(text):
    """
        Remove common-words
    """
    stopwords = list(set(nltk.corpus.stopwords.words('english')))
    avoiding_words = ['covid_19', 'covid', 'COVID', 'covid_', 'COVID_', 'CORONAVIRUS', 
                      'SARS_COV_2', 'coronavirus', 'coronaviruses', 
                      'sars_cov_2', 'conclusion', 'CONCLUSION', 'objective', 'OBJECTIVE', 'ABSTRACT', 'BACKGROUND'
                      'abstract', 'background', 'AUTHOR', 'DISCLOSURE', 'author', 'disclosure', 'title', 'TITLE']
    stopwords.extend(avoiding_words)
    sentences = nltk.sent_tokenize(text)
    new_text = []
    for word in text.split(' '):
        if word not in stopwords:
            new_text.append(word)
                
    cleaned_text = ' '.join(new_text)
 
    return cleaned_text

In [14]:
# text cleaning part1

text_cleaning = lambda x: cleaning_text(x)
english_covid_df['cleaned_text'] = english_covid_df['text'].apply(text_cleaning)
english_covid_df.drop(columns=['text'], inplace=True)
english_covid_df.rename(columns={'cleaned_text':'text'}, inplace=True)

In [15]:
# text cleaning part2

text_cleaning = lambda x: removing_stopwords(x)
english_covid_df['cleaned_text'] = english_covid_df['text'].apply(text_cleaning)
english_covid_df.drop(columns=['text'], inplace=True)
english_covid_df.rename(columns={'cleaned_text':'text'}, inplace=True)

## Drop articles after text cleaning

We exclude word in the text if the word is a digit, stop-words, or special charater during the text cleaning process. Therefore, some articles remain zero-word after the text cleaning process. We drop these zero-word articles from the corpus before we apply topic modelling.

In [16]:
# drop zero-word articles
english_covid_df.reset_index()
drop_index = []
for index, row in english_covid_df.iterrows():
    if len(row['text']) < 3:
        print(row['title'])
        drop_index.append(index)
print("{} of articles are dropped as the cleaned text is null".format(len(drop_index)))
preprocessed_df = english_covid_df.drop(drop_index, inplace=False)
preprocessed_df.shape

['What is Covid-19?']
['COVID-19 – where do we go from here?']
['The day after COVID-19']
['COVID-19 and the Eye']
['Where Is the ID in COVID-19?']
['The Day After Covid-19']
['Coronavirus and COVID-19']
['COVID-19 and the eye']
["What's new in COVID-19?"]
['NLP for COVID-19']
['How Do We End COVID-19?']
['The UK and Covid-19']
['COVID-19: Where Are We Now?']
['The BBC and Covid-19']
['COVID-19: the day after']
['COVID-19: where do we go from here?']
['COVID-19: A New Coronavirus']
['COVID-19 – Where should we go now?']
['CMR in the Era of COVID-19']
['Covid-19 and Covid-1619']
['COVID-19: A New Coronavirus']
['HIV in the age of COVID-19']
['XR in the era of COVID-19']
['COVID-19 (A): How Did We Get Here and What Do We Do Now?']
['How can we get out of COVID-19?']
['COVID-19: is the ACE2 just a foe?']
['Covid-19-vaccine-pfizer-biontech']
['Covid-19-Vaccine-Pfizer-Biontech']
['Covid-19-vaccine-pfizer-biontech']
['The 12-Lead ECG in COVID-19']
['Covid-19-vaccine-Pfizer-BioNTech']
['Covid

(413915, 73)

In [17]:
# save data preprocessing result
preprocessed_df.to_csv(OUTPUT_PATH + "preprocessed_data(all_data_type).tsv", sep='\t', encoding='utf-8', index=False)